<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Parsing-and-Basics" data-toc-modified-id="Parsing-and-Basics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parsing and Basics</a></span></li><li><span><a href="#Compare-to-other-genome-sequences" data-toc-modified-id="Compare-to-other-genome-sequences-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compare to other genome sequences</a></span></li><li><span><a href="#Extract-annotations-on-the-matching-genome-sequence" data-toc-modified-id="Extract-annotations-on-the-matching-genome-sequence-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract annotations on the matching genome sequence</a></span></li><li><span><a href="#Protein-level-analyses" data-toc-modified-id="Protein-level-analyses-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Protein level analyses</a></span><ul class="toc-item"><li><span><a href="#What's-next?" data-toc-modified-id="What's-next?-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>What's next?</a></span></li></ul></li></ul></div>

There is an unusual outbreak of diseases in your city. What causes it is unknown, but a sequence has been found in all the patients. Your are tasked with investigating the causes. Luckily, you have just completed a course in Python and are eager to try out what you've learned.

# Setup

You go ahead and import relevant packages from Biopython:

In [49]:
import os
import sys

import Bio
from Bio import SeqIO, SearchIO, Entrez, AlignIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

You then find the fasta file with the unknown sequence. Check out the file size first. If it's small enough, check it out in your editor before you load it into Python.

In [2]:
input_file = "unknown-sequence.fa"

To get an overview, we display the ID of every sequence in the file:

In [3]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


While this is not terribly helpful, we now know there is only one sequence.

# Parsing and Basics

We start out by parsing the file:

In [4]:
record = SeqIO.read(input_file, "fasta")

In [35]:
type(record)

Bio.SeqRecord.SeqRecord

This yields a Biopython SeqRecord object. See here for more details: https://biopython.org/wiki/SeqRecord

In [36]:
print(record)

ID: Unknown_sequence
Name: Unknown_sequence
Description: Unknown_sequence
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())


In [6]:
print("Sequence length (bp)", len(record))

Sequence length (bp) 29903


The sequence length is ~30Kb, if this sequence represents an individual organism then it is very small. Far too small for a typical eukaryote and in fact too short many microbes too (e.g. bacterial genomes are typically Mb). You have a dim feeling what this sequence may be...

In [7]:
print("GC content (%)", GC(record.seq))

GC content (%) 37.97277865097148


The GC content is 0.38, so the sequence is somewhat AT-rich, but within a 'normal' range.

# Compare to other genome sequences

You decide to run the sequence through Blast to see if you can find any close relatives. This may take ~10 minutes since we are doing an online search against many sequences. But for now it will do and it saves you from downloading the databases. (BioPython can run Blast for you locally as well.)

In [8]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

CPU times: user 1.51 s, sys: 297 ms, total: 1.8 s
Wall time: 29min 5s


That took long enough... You're eager to check out the results. Luckily, BioPython can also handle the parsing for you:

In [9]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")

In [10]:
print(blast_qresult)

Program: blastn (2.10.0+)
  Query: No (29903)
         definition line
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|1798174254|ref|NC_045512.2|  Severe acute respirator...
            1      1  gi|1819735426|gb|MT121215.1|  Severe acute respiratory ...
            2      1  gi|1805293633|gb|MT019531.1|  Severe acute respiratory ...
            3      1  gi|1820247323|dbj|LC529905.1|  Severe acute respiratory...
            4      1  gi|1818244627|gb|MT135044.1|  Severe acute respiratory ...
            5      1  gi|1818244605|gb|MT135042.1|  Severe acute respiratory ...
            6      1  gi|1818244594|gb|MT135041.1|  Severe acute respiratory ...
            7      1  gi|1805293611|gb|MT019529.1|  Severe acute respiratory ...
            8      1  gi|1802633808|gb|MN996528.1|  Severe acute res

Those descriptions are truncated, let's view them in full, for just the first 5 records

In [39]:
for hit in blast_qresult[:5]:
    print(hit.description)

Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SH01/human/2020/CHN, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate BetaCoV/Wuhan/IPBCAMS-WH-03/2019, complete genome
Severe acute respiratory syndrome coronavirus 2 TKYE6182_2020 RNA, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/235/human/2020/CHN, complete genome


Well that looks fairly conclusive, without doing any quantitative analyses, it's already very likely we have a coronavirus genome here, specifically SARS2-CoV-2 that was the cause of the COVID19 pandemic!

Let's have a look at the first result in a bit more detail to check some of the alignment metrics

In [12]:
first_hit = blast_qresult[0]

In [13]:
first_hit.description

'Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome'

In [14]:
first_hsp = first_hit[0]
print(first_hsp.evalue, first_hsp.bitscore)

0.0 53927.4


In [15]:
print(first_hsp.aln)

DNAAlphabet() alignment with 2 rows and 29903 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA gi|1798174254|ref|NC_045512.2|


Let's save the alignment to have a better look at it and for future reference:

In [55]:
AlignIO.write(first_hsp.aln, 'test', 'clustal')

1

# Extract annotations on the matching genome sequence

Let's extract a bit more structured meta-data on the top matching sequence homologous sequence using NCBI Entrez via Biopython to extract a GenBank file

In [74]:
print(first_hit.id)

gi|1798174254|ref|NC_045512.2|


In [17]:
NCBI_id = first_hit.id.split('|')[3]
NCBI_id

'NC_045512.2'

In [18]:
Entrez.email = "A.N.Other@example.com"  # Always tell NCBI who you are; promise there's no spam mails

In [19]:
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb")

In [20]:
genbank_record = SeqIO.read(handle, "genbank")

In [21]:
genbank_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', IUPACAmbiguousDNA()), id='NC_045512.2', name='NC_045512', description='Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=['BioProject:PRJNA485481'])

There's a lot of information in the genbank record if you know where to find it...

First you decide to double-check whether the virus has single- or double-stranded DNA or RNA:

In [22]:
print("Is it single or double stranded and a DNA or RNA virus?\n",
      genbank_record.annotations["molecule_type"])

Is it single or double stranded and a DNA or RNA virus?
 ss-RNA


That's odd. Your original sample was DNA. Any idea why?

You decide to investigate the Virus's full taxonomy:

In [56]:
genbank_record.annotations["taxonomy"]

['Viruses',
 'Riboviria',
 'Nidovirales',
 'Cornidovirineae',
 'Coronaviridae',
 'Orthocoronavirinae',
 'Betacoronavirus',
 'Sarbecovirus']

Lastly, you're curious who you have to thank for all this data:

In [24]:
for reference in genbank_record.annotations["references"]:
    print(reference)

What are the relevant references/labs who generated the data?

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.-M., Wang,W., Hu,Y., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Yuan,M.L., Zhang,Y.-L., Dai,F.-H., Liu,Y., Wang,Q.-M., Zheng,J.-J., Xu,L., Holmes,E.C. and Zhang,Y.-Z.
title: A novel coronavirus associated with a respiratory disease in Wuhan of Hubei province, China
journal: Unpublished
medline id: 
pubmed id: 
comment: 

location: [0:29903]
authors: 
consrtm: NCBI Genome Project
title: Direct Submission
journal: Submitted (17-JAN-2020) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA
medline id: 
pubmed id: 
comment: 

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.-M., Wang,W., Hu,Y., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Yuan,M.L., Zhang,Y.-L., Dai,F.-H., Liu,Y., Wang,Q.-M., Zheng,J.-J., Xu,L., Holmes,E.C. and Zhang,Y.-Z.
title: Direct Submission
journal: Submitted (05-JAN-2020) Shanghai Public Health Clinical Cent

That should be more than enough information for a bit.

Note that from this id, we could also find the [record here](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2/) on the NCBI website.

# Protein level analyses

You're also interested in the gene/protein sequences, not just the entire genome. Luckily for you, BioPython makes it easy to retrieve the protein coding sequences (CDSs) from the Genbank record:

In [58]:
genbank_record.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(29903), strand=1), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(265), strand=1), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(21555), strand=1), type='gene'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(265), ExactPosition(13468), strand=1), FeatureLocation(ExactPosition(13467), ExactPosition(21555), strand=1)], 'join'), type='CDS', location_operator='join'),
 SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(805), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(805), ExactPosition(2719), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(2719), ExactPosition(8554), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(8554), ExactPosition(10054), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(10054), ExactPosition(10972), strand=1), type='mat_

In [25]:
len(genbank_record.features)

47

In [26]:
{feature.type for feature in genbank_record.features}

{"3'UTR", "5'UTR", 'CDS', 'gene', 'mat_peptide', 'source', 'stem_loop'}

In [27]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
len(CDSs)

12

Let's look at the first protein and extract the underlying sequence

In [84]:
for cds in CDSs:
    print(cds.qualifiers["gene"])

['orf1ab']
['orf1ab']
['S']
['ORF3a']
['E']
['M']
['ORF6']
['ORF7a']
['ORF7b']
['ORF8']
['N']
['ORF10']


In [60]:
protein_seq = CDSs[0].qualifiers["translation"][0]
protein_seq = Seq(protein_seq)

In [85]:
CDSs[0].qualifiers["translation"]

['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQ

In [61]:
protein_seq

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')

The sequence should begin with a start codon, right?

In [62]:
protein_seq.startswith("M")

True

In case you needed a refresher on the codon table:

In [64]:
print(CodonTable.unambiguous_dna_by_id[1])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

However, we can't perform an exact "reverse translation" of course, since several amino acids are produced by the same codon. Note that if instead we started with the nucleotide sequence, then we could use Biopython's `.transcribe()` and `.translate()` functions to convert sequences from DNA to RNA and DNA to protein respectively.

In any case, whe can get the sequence length in amino acids:

In [65]:
len(protein_seq)

7096

That's quite a long protein for a virus. You decide to check the annotation:

In [34]:
CDSs[0].qualifiers["product"]

['orf1ab polyprotein']

So it looks like this is a polyprotein, which explains why it is a relatively long protein. Polyproteins are a typical feature of some viral genomes where smaller proteins are joined together, providing a particular organization of the viral proteome.

## What's next?

Logical next steps at the genome level might include building a multiple sequence alignment from many cornavirus genomes (checkout the Biopython wrapers/parsers for `Clustal` and `Mafft` and `Bio.Align`/`Bio.parirwise2` plus `Bio.AlignIO`), building a phylogeny with an external tool like [iq-tree](http://www.iqtree.org/) and then viewing the tree with `Bio.Phylo`, the [ete3 toolkit](http://etetoolkit.org/), or [Jalview](https://www.jalview.org/).

Other protein level analyses could involve including building protein trees, annotating the proteins and vizulisation (e.g. `Bio.Graphics`), doing evolutionary rate analyses (e.g. `Bio.Phylo.PAML `), looking at protein structure, clustering and much more.

These kind of analyses can provide useful biological and epidemiological information, very important for this coronavirus in an outbreak situation. For example, allowing tracking of how the outbreak spreads and indicating appropriate infection control measures, although caution in the interpretation of results is always required. See [Nextstrain](https://nextstrain.org/ncov) for an excellent resource of this kind.

Note, there is tons of other functionality in Biopython, this is just a very small fraction of the modules, see [Peter Cock's Biopython workshop](https://github.com/peterjc/biopython_workshop) and the extensive [official tutorial documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html).